<a href="https://colab.research.google.com/github/vignesh99/CS6300-Speech-Technology/blob/master/HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/1 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/2 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/3 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/6 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/7 /root
!cp -avr /content/gdrive/My\ Drive/HMM_CtsData/dev /root
!cp -avr /content/gdrive/My\ Drive/HMM_CtsData/test /root
!cp -avr /content/gdrive/My\ Drive/fp1.txt /root
!cp -avr /content/gdrive/My\ Drive/fp2.txt /root
!cp -avr /content/gdrive/My\ Drive/fp3.txt /root
!cp -avr /content/gdrive/My\ Drive/fp6.txt /root
!cp -avr /content/gdrive/My\ Drive/fp7.txt /root
!ls

/root
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
removed '/root/1'
'/content/gdrive/My Drive/HMM_IsoData/1' -> '/root/1'
removed '/root/2'
'/content/gdrive/My Drive/HMM_IsoData/2' -> '/root/2'
removed '/root/3'
'/content/gdrive/My Drive/HMM_IsoData/3' -> '/root/3'
removed '/root/6'
'/content/gdrive/My Drive/HMM_IsoData/6' -> '/root/6'
removed '/root/7'
'/content/gdrive/My Drive/HMM_IsoData/7' -> '/root/7'
removed '/root/dev/team15'
'/content/gdrive/My Drive/HMM_CtsData/dev/team15' -> '/root/dev/team15'
removed '/root/test/team15'
'/content/gdrive/My Drive/HMM_CtsData/test/team15' -> '/root/test/team15'
'/content/gdrive/My Drive/fp1.txt' -> '/root/fp1.txt'
'/content/gdrive/My Drive/fp2.txt' -> '/root/fp2.txt'
'/content/gdrive/My Drive/fp3.txt' -> '/root/fp3.txt'
'/content/gdrive/My Drive/fp6.txt' -> '/root/fp6.txt'
'/content/gdrive/My Drive/fp7.txt' -> '/root/fp7.txt'
1  6	fp1.txt  fp6.txt  one.test 

In [41]:
                                    #Test file
!cp -avr /root/1/dev/ra_1.mfcc /root
filex = "ra_1.mfcc"

'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'


In [42]:
                                    #Import required libraries
from pylab import *
import os                           #Going through folders
from scipy.cluster.vq import kmeans2
from google.colab import files
from sklearn.cluster import KMeans

In [43]:
def fileTofeature(fileMfcc,Nclust=15) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
                                    
    return Nf,features


In [44]:
label,ft = fileTofeature(filex)
label

121

In [45]:
                                    #Give the file with sequences for training data
def trainCentre(word,Nclust=15) :
    Nwin = []                       #No. of vectors in each train file
    count = 0
                                    #Go through required folders
    for subdir, dirs, files in os.walk(word):
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        if dirname == "train":
            for f in files:
                if f.endswith('.mfcc'):
                                    #Copy file (mfcc) to root and perform K-means
                    !cp -avr /root/$word/train/$f /root
                    foo,temp = fileTofeature(f)
                                    #Concatenate training feature vectors
                    if count == 0 :
                        features = temp
                        Nwin.append(foo)
                    else :
                        features = np.concatenate((features,temp))
                        Nwin.append(foo+Nwin[-1])
                    !rm $f
                    count+=1
                                    #Apply K-means on the vectors
    trainKM = KMeans(n_clusters=Nclust)
    trainKM.fit(features)

    return Nwin,features,trainKM


In [46]:
def seqDwnld(seq,fileWord) :
    with open(fileWord,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in seq))

    return None

In [47]:
                                    #Give the file with sequences for training data
def trainSeq(Nwin,features,trainKM,fileWord="check.txt") :
    seq = []
    count = 0
                                    #Go through required folders
    for i in range(0,len(Nwin)) :
                                    #Get each features from all-features-array
        if i != 0:
            temp = features[Nwin[i-1]:Nwin[i]]
        else : 
            temp = features[:Nwin[0]]
                                    #Perform Kmeans on predefined centres
        label = trainKM.predict(temp)
        seq.append(label)

        if i==2 :
            print(label)

    seq = array(seq)

    seqDwnld(seq,fileWord)

    return None


In [48]:
!ls

1  6	fp1.txt  fp6.txt  one.test   seven.test  test	     two.seq
2  7	fp2.txt  fp7.txt  ra_1.mfcc  six.seq	 three.seq   two.test
3  dev	fp3.txt  one.seq  seven.seq  six.test	 three.test


In [68]:
# Nw,feat,oneKM = trainCentre("1")
# trainSeq(Nw,feat,oneKM,"one.seq")

# Nw,feat,twoKM = trainCentre("2")
# trainSeq(Nw,feat,twoKM,"two.seq")

Nw,feat,threeKM = trainCentre("3")
trainSeq(Nw,feat,threeKM,"three.seq")

# Nw,feat,sixKM = trainCentre("6")
# trainSeq(Nw,feat,sixKM,"six.seq")

Nw,feat,sevenKM = trainCentre("7")
trainSeq(Nw,feat,sevenKM,"seven.seq")


Directory: 3
Directory: train
'/root/3/train/ac_3.mfcc' -> '/root/ac_3.mfcc'
'/root/3/train/ag_3.mfcc' -> '/root/ag_3.mfcc'
'/root/3/train/ai_3.mfcc' -> '/root/ai_3.mfcc'
'/root/3/train/an_3.mfcc' -> '/root/an_3.mfcc'
'/root/3/train/bh_3.mfcc' -> '/root/bh_3.mfcc'
'/root/3/train/bi_3.mfcc' -> '/root/bi_3.mfcc'
'/root/3/train/br_3.mfcc' -> '/root/br_3.mfcc'
'/root/3/train/ca_3.mfcc' -> '/root/ca_3.mfcc'
'/root/3/train/cg_3.mfcc' -> '/root/cg_3.mfcc'
'/root/3/train/cl_3.mfcc' -> '/root/cl_3.mfcc'
'/root/3/train/cm_3.mfcc' -> '/root/cm_3.mfcc'
'/root/3/train/dc_3.mfcc' -> '/root/dc_3.mfcc'
'/root/3/train/dg_3.mfcc' -> '/root/dg_3.mfcc'
'/root/3/train/ea_3.mfcc' -> '/root/ea_3.mfcc'
'/root/3/train/ec_3.mfcc' -> '/root/ec_3.mfcc'
'/root/3/train/ee_3.mfcc' -> '/root/ee_3.mfcc'
'/root/3/train/eg_3.mfcc' -> '/root/eg_3.mfcc'
'/root/3/train/ei_3.mfcc' -> '/root/ei_3.mfcc'
'/root/3/train/ek_3.mfcc' -> '/root/ek_3.mfcc'
'/root/3/train/es_3.mfcc' -> '/root/es_3.mfcc'
'/root/3/train/hg_3.mfcc' -> '

In [69]:
                                    #Download the train sequence files
# files.download("one.seq")
#files.download("two.seq")
files.download("three.seq")
# files.download("six.seq")
files.download("seven.seq")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
!ls

1  6	fp1.txt  fp6.txt  one.test   seven.test  test	     two.seq
2  7	fp2.txt  fp7.txt  ra_1.mfcc  six.seq	 three.seq   two.test
3  dev	fp3.txt  one.seq  seven.seq  six.test	 three.test


In [53]:
def testKmeans(fileMfcc,trainKM,Nclust=15,iters=1) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
    
                                    #Apply K-means on the vectors
    if len(features) == Nf and len(features[0]) == Nc :
        label = trainKM.predict(features)

    return label


In [54]:
                                    #Give the file with sequences for testing data
def testSeq(trainKM,fileWord="check.txt",folders=None) :
    if folders is None :
        folders = array(["1","2","3","6","7"])
    seq = []
    count = np.zeros(5)
    for word in folders :    
                                        #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "dev":
                for f in files:
                    if f.endswith(".mfcc"):
                                        #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/dev/$f /root
                        label = testKmeans(f,trainKM)
                        seq.append(label)
                        !rm $f
                        count[np.where(word == folders)[0][0]] += 1
        if word != "1" :
            count[np.where(word == folders)[0][0]] = count[np.where(word == folders)[0][0]] + count[np.where(word == folders)[0][0]-1]

    print(count)
    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return count


In [77]:

# count = testSeq(oneKM,"one.test")
#count = testSeq(twoKM,"two.test")
count = testSeq(threeKM,"three.test")
# count = testSeq(sixKM,"six.test")
#count = testSeq(sevenKM,"seven.test")

Directory: 1
Directory: train
Directory: dev
'/root/1/dev/mk_1.mfcc' -> '/root/mk_1.mfcc'
'/root/1/dev/mm_1.mfcc' -> '/root/mm_1.mfcc'
'/root/1/dev/ms_1.mfcc' -> '/root/ms_1.mfcc'
'/root/1/dev/mw_1.mfcc' -> '/root/mw_1.mfcc'
'/root/1/dev/nc_1.mfcc' -> '/root/nc_1.mfcc'
'/root/1/dev/ng_1.mfcc' -> '/root/ng_1.mfcc'
'/root/1/dev/nh_1.mfcc' -> '/root/nh_1.mfcc'
'/root/1/dev/pe_1.mfcc' -> '/root/pe_1.mfcc'
'/root/1/dev/pk_1.mfcc' -> '/root/pk_1.mfcc'
'/root/1/dev/pm_1.mfcc' -> '/root/pm_1.mfcc'
'/root/1/dev/pp_1.mfcc' -> '/root/pp_1.mfcc'
'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'
Directory: 2
Directory: train
Directory: dev
'/root/2/dev/mk_2.mfcc' -> '/root/mk_2.mfcc'
'/root/2/dev/mm_2.mfcc' -> '/root/mm_2.mfcc'
'/root/2/dev/ms_2.mfcc' -> '/root/ms_2.mfcc'
'/root/2/dev/mw_2.mfcc' -> '/root/mw_2.mfcc'
'/root/2/dev/nc_2.mfcc' -> '/root/nc_2.mfcc'
'/root/2/dev/ng_2.mfcc' -> '/root/ng_2.mfcc'
'/root/2/dev/nh_2.mfcc' -> '/root/nh_2.mfcc'
'/root/2/dev/pe_2.mfcc' -> '/root/pe_2.mfcc'
'/root/2/d

In [78]:
                                    #Download the test sequence files
# files.download("one.test")
#files.download("two.test")
files.download("three.test")
# files.download("six.test")
#files.download("seven.test")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
count = testSeq(oneKM,"one.test")

In [73]:
def testProb(fileProb) :
    prob = np.genfromtxt(fileProb,delimiter=' ',dtype=str)
    prob = prob.astype(float)

    return prob

In [74]:
!ls

1  6	fp1.txt  fp6.txt  one.test    six.seq	three.seq   two.test
2  7	fp2.txt  fp7.txt  seven.seq   six.test	three.test
3  dev	fp3.txt  one.seq  seven.test  test	two.seq


In [75]:
def accModel(count) :

    error = np.zeros(5)
    prob1 = testProb("fp1.txt")
    allprob = np.zeros((len(prob1),5))
    allprob[:,0] = prob1
    
    allprob[:,1] = testProb("fp2.txt")
    allprob[:,2] = testProb("fp3.txt")
    allprob[:,3] = testProb("fp6.txt")
    allprob[:,4] = testProb("fp7.txt")
    
    estimate = np.argmax(allprob,axis=1)
    print(estimate)
    for i in range(0,5) :
        if i==0:
            error[i] = len(np.where(estimate[:count[i]]!=i)[0])
        else :
            error[i] = len(np.where(estimate[count[i-1]:count[i]]!=i)[0])

    print(error)
    acc = 1-(sum(error)/count[-1])
    
    return acc

In [76]:
#acc = accModel(array([12,24,36,48,60]))
acc = accModel(count.astype(int))
print(acc)

[0 0 0 0 0 0 0 0 0 0 0 0 4 1 1 1 1 3 1 0 1 1 1 1 2 4 4 0 2 3 2 2 3 2 4 4 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 3 4 4 4 4 4 4]
[0. 3. 7. 0. 1.]
0.8166666666666667
